In [73]:
from bs4 import BeautifulSoup
import requests
import os
from urllib.parse import urljoin
import re
import pandas as pd

In [78]:
core_url = "https://www.airlinequality.com/airline-reviews"

N_REVIEWS = 50

airlines = ["ryanair", "lufthansa", "air-france", "aeroflot-russian-airlines", "turkish-airlines", "wizz-air", "easyjet", "tui-airways", "norwegian","tap-portugal","emirates","jetblue-airways","united-airlines","air-canada","singapore-airlines"]



In [79]:
#reference from Skytrax
#https://www.airlinequality.com/

airline_reviews = pd.DataFrame(columns=['airline', 'comment', 'title', 'rating', 'user_country', 'date', 'recommend'])

In [80]:
for airline in airlines:
    # url of airline
    skytrax_url = os.path.join(core_url, airline, "?sortby=post_date%3ADesc&pagesize=" + str(N_REVIEWS))
    # get HTML
    response = requests.get(skytrax_url)
    print(response,skytrax_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # get HTML of section which includes reviews
    reviews = soup.find('section', class_='layout-section layout-2 closer-top')
    # get fields
    reviews_comment = [tag.get_text() for tag in soup.select('div[class="text_content"]')]
    reviews_title = [tag.get_text() for tag in soup.select('h2[class="text_header"]')][:N_REVIEWS]
    reviews_rating = [int(tag.get_text()) for tag in reviews.select('span[itemprop="ratingValue"]')]
    reviews_user_country = [re.findall(r'\(([A-Za-z ]+)\)', tag.get_text())[0] for tag in reviews.select('h3[class="text_sub_header userStatusWrapper"]')]
    reviews_date = [tag.get_text() for tag in reviews.select('time[itemprop="datePublished"]')]
    reviews_recommend = [tag.get_text() for tag in reviews.select('td[class^="review-value rating-"]')]
    
    df_temp = pd.DataFrame({'comment': reviews_comment, 'title': reviews_title, 'rating': reviews_rating, 'user_country': reviews_user_country, 'date': reviews_date, 'recommend': reviews_recommend})
    df_temp['airline'] = airline
    airline_reviews = pd.concat([airline_reviews, df_temp])

<Response [200]> https://www.airlinequality.com/airline-reviews/ryanair/?sortby=post_date%3ADesc&pagesize=50
<Response [200]> https://www.airlinequality.com/airline-reviews/lufthansa/?sortby=post_date%3ADesc&pagesize=50
<Response [200]> https://www.airlinequality.com/airline-reviews/air-france/?sortby=post_date%3ADesc&pagesize=50
<Response [200]> https://www.airlinequality.com/airline-reviews/aeroflot-russian-airlines/?sortby=post_date%3ADesc&pagesize=50
<Response [200]> https://www.airlinequality.com/airline-reviews/turkish-airlines/?sortby=post_date%3ADesc&pagesize=50
<Response [200]> https://www.airlinequality.com/airline-reviews/wizz-air/?sortby=post_date%3ADesc&pagesize=50
<Response [200]> https://www.airlinequality.com/airline-reviews/easyjet/?sortby=post_date%3ADesc&pagesize=50
<Response [200]> https://www.airlinequality.com/airline-reviews/tui-airways/?sortby=post_date%3ADesc&pagesize=50
<Response [200]> https://www.airlinequality.com/airline-reviews/norwegian/?sortby=post_date

In [81]:
airline_reviews.to_csv('cleaned_data/airline-reviews.csv')

In [82]:
airline_reviews.columns = ['airline_id', 'comment', 'title', 'rating', 'user_country_id', 'date', 'recommend']
airline_reviews.reset_index(drop=True, inplace=True)

In [83]:
airline_reviews.head()

,airline_id,comment,title,rating,user_country_id,date,recommend
0,ryanair,Not Verified | Do not fly Ryanair if you have...,"""they made our experience so stressful""",1,United Kingdom,6th April 2022,no
1,ryanair,✅ Trip Verified | I tried to place booking at...,"""almost two weeks and it still not refunded""",1,Ukraine,4th April 2022,no
2,ryanair,✅ Trip Verified | Terrible customer service a...,"""Terrible customer service""",1,United States,3rd April 2022,no
3,ryanair,✅ Trip Verified | I bought a 10 kg roll on ha...,"""I chose not to fly""",1,Spain,29th March 2022,no
4,ryanair,Not Verified | Plane is very dirty and some o...,"""Plane is very dirty""",3,Finland,25th March 2022,no


In [84]:
airline_reviews.to_parquet('cleaned_data/New_airline-reviews.parquet')
airline_reviews.to_csv('cleaned_data/New_airline-reviews.csv')